In [1]:
# Ячейка 1: ПРИНУДИТЕЛЬНОЕ ОБНОВЛЕНИЕ БИБЛИОТЕК
!apt-get update -qq
!apt-get install -y -qq djvulibre-bin
# Флаг -U обязателен, чтобы получить доступ к gemini-1.5-flash
!pip install -U -q google-generativeai telethon pymupdf tqdm pillow
!echo "✅ Библиотеки ОБНОВЛЕНЫ. Теперь модель должна быть видна."

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package djvulibre-bin.
(Reading database ... 128639 files and directories currently installed.)
Preparing to unpack .../djvulibre-bin_3.5.28-2ubuntu0.22.04.1_amd64.deb ...
Unpacking djvulibre-bin (3.5.28-2ubuntu0.22.04.1) ...
Selecting previously unselected package libexiv2-27:amd64.
Preparing to unpack .../libexiv2-27_0.27.5-3ubuntu1_amd64.deb ...
Unpacking libexiv2-27:amd64 (0.27.5-3ubuntu1) ...
Selecting previously unselected package libgraphicsmagick-q16-3.
Preparing to unpack .../libgraphicsmagick-q16-3_1.4+really1.3.38-1ubuntu0.1_amd64.deb ...
Unpacking libgraphicsmagick-q16-3 (1.4+really1.3.38-1ubuntu0.1) ...
Selecting previously unselected package libgraphicsmagick++-q16-12.
Preparing to unpack .../libgraphicsmagick++-q16-12_1.4+really1.3.38-1ubuntu0.1_amd64.de

In [ ]:
# Ячейка Диагностики
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

print("🔍 Сканирование доступных моделей...")
try:
    user_secrets = UserSecretsClient()
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GENAI_KEY)
    
    available_models = []
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"   ✅ Доступна: {m.name}")
            available_models.append(m.name)
            
    if not available_models:
        print("❌ Нет доступных моделей! Проверьте API Key.")
    else:
        print(f"\nВсего найдено: {len(available_models)}")

except Exception as e:
    print(f"❌ Критическая ошибка: {e}")
    print("💡 СОВЕТ: Нажмите в меню 'Run' -> 'Restart Session', чтобы обновить библиотеки.")

In [2]:
# === ЯЧЕЙКА 2: V57.1 (FULL META + JSON REPAIR) ===
import os
import time
import json
import asyncio
import shutil
import re
import fitz  # PyMuPDF
import subprocess
import google.generativeai as genai
from PIL import Image
from datetime import datetime
from telethon import TelegramClient, events, functions, types
from telethon.sessions import MemorySession
from kaggle_secrets import UserSecretsClient
from tqdm.notebook import tqdm
from google.api_core.exceptions import ResourceExhausted, ServiceUnavailable, InternalServerError

# === SAFETY ===
Image.MAX_IMAGE_PIXELS = None 

# === SETTINGS ===
SOURCE_USERNAME = "sendDoc39"
# Стратегия: пробуем 5, при ошибке 3, при совсем ошибке 1
BATCH_STRATEGY = [5, 3, 1]  

GLOBAL_REQUEST_DELAY = 30.0  

ZOOM_FACTOR = 2.0           
MARKER_EMOJI = "✅"
MAX_EMPTY_GAP = 5
SAFETY_LIMIT = 500

DJVU_TARGET_SIZE = 2560     
CROP_PADDING = 15           
SNAP_THRESHOLD = 30         

# --- PRODUCTION SETTINGS ---
DEBUG_PAGE_LIMIT = 5      
# ---------------------------

user_secrets = UserSecretsClient()
TEMP_DIR = "./temp_processing"
CROPS_DIR = "./temp_crops"

LOG_BUFFER = []

def log(text):
    tqdm.write(text)
    LOG_BUFFER.append(text)

async def send_report(client, dest_entity):
    report_text = "\n".join(LOG_BUFFER)
    if len(report_text) > 4000: report_text = report_text[-3900:]
    try:
        if dest_entity: await client.send_message(dest_entity, f"📝 **Report:**\n\n...{report_text}")
    except: pass

# === JSON REPAIR KIT ===
def repair_json_content(text):
    """
    Лечит ошибку 'Invalid \escape', экранируя одиночные слеши,
    но сохраняя валидные JSON-последовательности (\n, \t, \", \\).
    """
    # Паттерн находит backslash, за которым НЕ следует валидный escape-символ
    # Valid JSON escapes: ", \, /, b, f, n, r, t, u
    pattern = r'\\(?![/u"bfnrt\\])'
    return re.sub(pattern, r'\\\\', text)

# === MODEL MANAGER ===
class ModelRotator:
    def __init__(self, api_keys):
        if isinstance(api_keys, str): self.api_keys = [api_keys]
        else: self.api_keys = api_keys
        self.current_key_idx = 0
        self._configure_current_key()
        self.last_request_ts = 0
        
        # Priority Models
        self.models_list = [
            "models/gemini-2.5-flash",      # Priority 1
            "models/gemini-2.0-flash-exp",  # Priority 2
            "models/gemini-1.5-flash",      # Priority 3
        ]
        log(f"🧠 Models initialized. Main: {self.models_list[0]}. Delay: {GLOBAL_REQUEST_DELAY}s.")

    def _configure_current_key(self):
        genai.configure(api_key=self.api_keys[self.current_key_idx])

    async def _enforce_global_pace(self):
        now = time.time()
        elapsed = now - self.last_request_ts
        if elapsed < GLOBAL_REQUEST_DELAY:
            wait_time = GLOBAL_REQUEST_DELAY - elapsed + 1
            await asyncio.sleep(wait_time)
        self.last_request_ts = time.time()

    async def generate_content(self, content):
        for i, model_name in enumerate(self.models_list):
            await self._enforce_global_pace()
            
            # Low temp for structural adherence
            model = genai.GenerativeModel(
                model_name=model_name,
                safety_settings=[{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}],
                generation_config={"response_mime_type": "application/json", "temperature": 0.0}
            )
            
            try:
                log(f"      📡 [{model_name}] Sending...")
                t_start = time.time()
                response = await asyncio.to_thread(model.generate_content, content)
                dur = time.time() - t_start
                
                usage = {"in": 0, "out": 0}
                if response.usage_metadata:
                    usage["in"] = response.usage_metadata.prompt_token_count
                    usage["out"] = response.usage_metadata.candidates_token_count
                
                return response.text, usage, model_name, dur
            
            except (ResourceExhausted, ServiceUnavailable, InternalServerError) as e:
                log(f"      ⚠️ Limit/Error on {model_name}.")
                if i == len(self.models_list) - 1:
                    log("      ❌ ALL MODELS EXHAUSTED. Aborting execution.")
                    raise e 
                log("      ➡️ Switching to backup model...")
                continue
                
            except Exception as e:
                if "429" in str(e):
                    log(f"      ⚠️ 429 Too Many Requests on {model_name}.")
                    if i == len(self.models_list) - 1:
                        log("      ❌ ALL MODELS EXHAUSTED (429). Aborting.")
                        raise e
                    continue
                log(f"      ❌ Unexpected Error ({model_name}): {e}")
                raise e

        raise Exception("Unexpected logic flow in ModelRotator")

# === INIT ===
model_rotator = None
try:
    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    model_rotator = ModelRotator(GENAI_KEY)
except Exception as e: log(f"❌ Init Error: {e}")

# === AI LOGIC (UPDATED PROMPT V57.1) ===
def get_gemini_prompt(is_first_batches, previous_context=None):
    base_prompt = """
    TASK: OCR & Layout Analysis (Spread vs Journal Page) with Rich Formatting.
    """
    
    if previous_context:
        base_prompt += f"""
    CONTEXT FROM PREV BATCH: "...{previous_context}..."
    """

    base_prompt += """
    OUTPUT FORMAT (JSON):
    {
      "pages": [
        {
           "book_page_num": "string",         // PRINTED page number.
           "classification_code": "string",   // UDC/BBC codes (e.g. "УДК...").
           "main_text": "string",             // Markdown text with headers/bold/italic.
           "footnotes": "string",             // Isolated footnotes.
           "marginalia": "string",            // Headers/Running titles.
           "media_objects": [
              { "id": "string", "coordinates": [y,x,y,x], "type": "string", "caption_ru": "string" }
           ]
        }
      ],
    """
    
    if is_first_batches:
        base_prompt += """
      "bibliographic_data": {
          "title": "string",
          "authors": ["string"],
          "co_authors": ["string"],
          "publisher": "string",
          "year": "string",
          "isbn": "string",
          "document_type": "string (book, article, official_document, archive_record, other)",
          "historical_period_description": "string (на русском)",
          "historical_year_start": "int or null",
          "historical_year_end": "int or null",
          "summary": "string (на русском)"
      }
    }
    """
    else:
        base_prompt += '  "batch_metadata": { "status": "cont" }\n}'

    # --- RULES V57.1 ---
    base_prompt += """
    CRITICAL RULES:
    
    1. **LAYOUT & ZONES**:
       - Check for **Single Page** vs **Spread** (2 pages).
       - If **Single Page** (Journal Style): Detect horizontal zones (Top Article end / Bottom Article start). Use `***` separator.
       - Do not merge columns across different articles.

    2. **RICH MARKDOWN FORMATTING (HIERARCHY)**:
       - **Headers**: Detect font size/boldness. Use `##` for Main Titles, `###` for Subtitles.
       - **Emphasis**: Use `**bold**` for bold text. Use `*italic*` for italics.
       - **Structure**: Preserve paragraph breaks.

    3. **METADATA**:
       - Extract full bibliographic data if available (first batch).
       - Use **Russian language** for descriptions and summaries.
       - If a value is unknown, use `null` or empty string.

    4. **FOOTNOTES**:
       - CUT footnote text from main body and PASTE into `footnotes`.
       
    5. **JSON SAFETY**:
       - Do not use raw backslashes inside strings unless escaping. Avoid LaTeX macros if possible, use Unicode.
    """
    return base_prompt

async def process_single_batch(image_paths, page_nums, prev_ctx_text):
    if not model_rotator: raise Exception("No models")
    
    is_first_batch = (page_nums[0] == 1)
    
    prompt = get_gemini_prompt(is_first_batch, prev_ctx_text)
    content = [prompt]
    loaded_imgs = []
    
    for p in image_paths:
        img = Image.open(p)
        w, h = img.size
        if w > 3072 or h > 3072:
            img.thumbnail((3072, 3072), Image.Resampling.LANCZOS)
        loaded_imgs.append(img)
        content.append(img)
        
    text_resp, usage, model_name, duration = await model_rotator.generate_content(content)
    
    clean_text = text_resp.replace("```json", "").replace("```", "").strip()
    if clean_text.startswith("json"): clean_text = clean_text[4:].strip()
    match = re.search(r'\{.*\}', clean_text, re.DOTALL)
    if match: clean_text = match.group(0)

    try:
        # 1. Попытка парсинга "как есть"
        data = json.loads(clean_text) 
    except Exception as e1:
        # 2. Попытка лечения JSON (backslashes)
        log(f"      ⚠️ JSON Error 1: {e1}. Repairing...")
        repaired_text = repair_json_content(clean_text)
        try:
            data = json.loads(repaired_text)
            log("      🛠️ Repair Successful!")
        except Exception as e2:
            log(f"      ❌ Repair Failed: {e2}. Raw text len: {len(clean_text)}")
            raise Exception("JSON Parsing Failed completely") 
    
    batch_meta = data.get("bibliographic_data", data.get("batch_metadata", None))
    return data.get("pages", []), loaded_imgs, usage, model_name, duration, batch_meta

def crop_images(pil_imgs, pages_data):
    crops = []
    if not os.path.exists(CROPS_DIR): os.makedirs(CROPS_DIR, exist_ok=True)
    
    for i, page_data in enumerate(pages_data):
        if i >= len(pil_imgs): src = pil_imgs[-1]
        else: src = pil_imgs[i]
        
        media_list = page_data.get("media_objects", [])
        w, h = src.size
        
        # Cover Guard
        is_cover_page = False
        cover_object = None
        for m in media_list:
            if m.get("type") == "cover" or m.get("id") == "book_cover":
                is_cover_page = True
                cover_object = m
                break
        
        if is_cover_page and cover_object:
            cover_object["coordinates"] = [0, 0, 1000, 1000]
            cover_object["id"] = "book_cover"
            media_list = [cover_object]
        
        for meta in media_list:
            try:
                ymin, xmin, ymax, xmax = meta["coordinates"]
                left = (xmin/1000)*w - CROP_PADDING
                top = (ymin/1000)*h - CROP_PADDING
                right = (xmax/1000)*w + CROP_PADDING
                bottom = (ymax/1000)*h + CROP_PADDING
                
                left = max(0, left); top = max(0, top)
                right = min(w, right); bottom = min(h, bottom)
                
                if (right-left < 30) or (bottom-top < 30): continue
                
                crop = src.crop((left, top, right, bottom))
                
                raw_id = meta.get("id", f"media_{int(time.time())}_{i}")
                if meta.get("type") == "cover": media_id = "book_cover"
                else: media_id = re.sub(r'[^a-zA-Z0-9_]', '', raw_id)
                
                c_name = f"{media_id}.png"
                c_path = os.path.join(CROPS_DIR, c_name)
                crop.save(c_path)
                
                caption = f"{meta.get('type', 'img').upper()}: {meta.get('caption_ru', '')}"
                crops.append((c_path, caption, media_id))
            except: pass
    return crops

# === TELEGRAM & UTILS ===
async def get_last_id(client, entity):
    try:
        full = await client(functions.channels.GetFullChannelRequest(entity))
        match = re.search(r"\[LastID:\s*(\d+)\]", full.full_chat.about or "")
        if match: return int(match.group(1))
    except: pass
    return None

async def update_last_id(client, entity, new_id):
    try:
        full = await client(functions.channels.GetFullChannelRequest(entity))
        about = full.full_chat.about or ""
        tag = f"[LastID: {new_id}]"
        if "[LastID:" in about: new_about = re.sub(r"\[LastID:\s*\d+\]", tag, about)
        else: new_about = f"{about}\n\n{tag}".strip()
        if new_about != about: await client(functions.messages.EditChatAboutRequest(peer=entity, about=new_about))
    except: pass

async def is_already_processed(message):
    try:
        if not message.reactions: return False
        for r in message.reactions.results:
            emoji = r.reaction.emoticon if hasattr(r.reaction, 'emoticon') else str(r.reaction)
            if emoji == MARKER_EMOJI and r.chosen: return True
    except: pass
    return False

async def mark_processed(client, msg):
    try:
        await client(functions.messages.SendReactionRequest(peer=msg.peer_id, msg_id=msg.id, reaction=[types.ReactionEmoji(emoticon=MARKER_EMOJI)]))
    except: pass

def render_pdf(path):
    doc = fitz.open(path)
    mat = fitz.Matrix(ZOOM_FACTOR, ZOOM_FACTOR)
    limit = len(doc)
    if DEBUG_PAGE_LIMIT:
        limit = min(len(doc), DEBUG_PAGE_LIMIT)
    for i in range(limit):
        page = doc.load_page(i)
        pix = page.get_pixmap(matrix=mat, alpha=False)
        out = os.path.join(TEMP_DIR, f"page_{i+1:04d}.png")
        pix.save(out)
        yield out, i+1, len(doc)
    doc.close()

def render_djvu(path):
    safe = os.path.join(TEMP_DIR, "temp.djvu")
    shutil.move(path, safe)
    try:
        subprocess.run([
            'ddjvu', '-format=tiff', f'-size={DJVU_TARGET_SIZE}x{DJVU_TARGET_SIZE}', 
            '-eachpage', safe, os.path.join(TEMP_DIR, "p_%04d.tif")
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.PIPE)
    except Exception as e:
        log(f"❌ DjVu Render Error: {e}")
        return

    files = sorted([f for f in os.listdir(TEMP_DIR) if f.endswith(".tif")])
    if DEBUG_PAGE_LIMIT: files = files[:DEBUG_PAGE_LIMIT]
    for i, f in enumerate(tqdm(files, desc="Conv", unit="pg", leave=False)):
        full = os.path.join(TEMP_DIR, f)
        try:
            with Image.open(full) as img:
                png = full.replace(".tif", ".png")
                img.save(png, "PNG")
                yield png, i+1, len(files)
            os.remove(full)
        except: pass
    if os.path.exists(safe): os.remove(safe)

# === MAIN ===
async def main_logic():
    log(f"🚀 **Launch V57.1 (Full Meta + JSON Repair):** {datetime.now().strftime('%H:%M:%S')}")
    
    try:
        API_ID = int(user_secrets.get_secret("TELEGRAM_API_ID"))
        API_HASH = user_secrets.get_secret("TELEGRAM_API_HASH")
        BOT_TOKEN = user_secrets.get_secret("TELEGRAM_BOT_TOKEN")
        DEST_ID = int(user_secrets.get_secret("DEST_CHANNEL_ID"))
    except: return log("❌ Secrets missing.")

    for d in [TEMP_DIR, CROPS_DIR]:
        if not os.path.exists(d): os.makedirs(d, exist_ok=True)

    client = TelegramClient(MemorySession(), API_ID, API_HASH)
    await client.start(bot_token=BOT_TOKEN)
    log("✅ Telegram OK.")
    if not model_rotator: return await client.disconnect()

    try:
        src_ent = await client.get_entity(SOURCE_USERNAME)
        dst_ent = await client.get_entity(DEST_ID)
    except: return await client.disconnect()

    start_id = await get_last_id(client, src_ent) or 1
    curr_id = start_id + 1
    processed = 0
    empty_streak = 0
    t_in, t_out = 0, 0
    pbar = tqdm(total=SAFETY_LIMIT, desc="Scan", unit="msg")

    while processed < SAFETY_LIMIT:
        if empty_streak >= MAX_EMPTY_GAP: log("🏁 End."); break
        try:
            msgs = await client.get_messages(src_ent, ids=[curr_id])
            msg = msgs[0] if msgs else None
            
            if not msg or isinstance(msg, types.MessageEmpty):
                empty_streak += 1; curr_id += 1; pbar.update(1); continue
            
            if await is_already_processed(msg):
                await update_last_id(client, src_ent, msg.id)
                curr_id += 1; pbar.update(1); continue

            fname = getattr(msg.file, 'name', None)
            if not fname: curr_id += 1; pbar.update(1); continue
            ext = fname.split('.')[-1].lower()
            if ext not in ['pdf', 'djvu']: curr_id += 1; pbar.update(1); continue

            empty_streak = 0
            log(f"\n📘 **{fname}** (ID: {msg.id})")
            fpath = os.path.join(TEMP_DIR, fname)
            await msg.download_media(file=fpath)
            
            iter_pages = render_pdf(fpath) if ext == 'pdf' else render_djvu(fpath)
            
            # === FULL METADATA STRUCT V57 ===
            full_bibliographic_data = {
                "isbn": "",
                "title": "",
                "authors": [],
                "co_authors": [],
                "publisher": "",
                "year": "",
                "document_type": "",
                "historical_period_description": "",
                "historical_year_start": "",
                "historical_year_end": "",
                "summary": "",
                "source_telegram_channel_id": str(dst_ent.id),
                "source_origin_msg_id": str(msg.id)
            }
            
            global_text_blocks = []      
            global_resources = []        
            global_footnotes = []
            global_marginalia = set()
            
            last_batch_tail_text = None 
            current_batch_size = BATCH_STRATEGY[0]
            
            generated_pages_buffer = []
            for p in iter_pages: generated_pages_buffer.append(p)
            
            cursor = 0
            total_pages = len(generated_pages_buffer)
            book_failed = False

            while cursor < total_pages:
                end = min(cursor + current_batch_size, total_pages)
                current_slice = generated_pages_buffer[cursor : end]
                slice_paths = [x[0] for x in current_slice]
                slice_idxs = [x[1] for x in current_slice]
                
                log(f"   🔄 Batch {slice_idxs[0]}-{slice_idxs[-1]} (Size: {len(slice_paths)})...")
                
                try:
                    pages_list, imgs, usg, m_name, duration, meta = await process_single_batch(
                        slice_paths, slice_idxs, last_batch_tail_text
                    )
                    
                    log(f"      ✅ [{m_name}] OK in {duration:.1f}s")
                    t_in += usg.get("in",0); t_out += usg.get("out",0)
                    
                    # === BATCH UPLOAD ===
                    page_msg_ids = {} 
                    ALBUM_CHUNK_SIZE = 10
                    for i in range(0, len(slice_paths), ALBUM_CHUNK_SIZE):
                        sub_paths = slice_paths[i : i + ALBUM_CHUNK_SIZE]
                        sub_idxs = slice_idxs[i : i + ALBUM_CHUNK_SIZE]
                        try:
                            sent_msgs = await client.send_file(dst_ent, file=sub_paths, force_document=True)
                            if not isinstance(sent_msgs, list): sent_msgs = [sent_msgs]
                            for sent_m, p_idx in zip(sent_msgs, sub_idxs):
                                page_msg_ids[p_idx] = sent_m.id
                            await asyncio.sleep(2) 
                        except: pass
                    # ====================

                    if meta and isinstance(meta, dict):
                        for k, v in meta.items():
                            if k in full_bibliographic_data and v:
                                # Overwrite empty fields, don't overwrite existing valid data if new one is partial
                                curr = full_bibliographic_data[k]
                                if not curr: full_bibliographic_data[k] = v

                    crops = crop_images(imgs, pages_list)
                    for cp_path, cp_cap, cp_id in crops:
                        msg_link = "local_error"
                        if os.path.exists(cp_path):
                            try:
                                c_msg = await client.send_file(dst_ent, cp_path, caption=f"🖼 {cp_cap}\n🆔 {cp_id}", force_document=False)
                                msg_link = f"tg://msg_id?id={c_msg.id}"
                            except: pass
                            finally:
                                if os.path.exists(cp_path): os.remove(cp_path)
                                await asyncio.sleep(2)
                        global_resources.append({"id": cp_id, "caption": cp_cap, "link": msg_link})

                    current_img_idx = 0
                    for i, page_data in enumerate(pages_list):
                        if i < len(slice_idxs): container_p = slice_idxs[i]
                        else: container_p = slice_idxs[-1]
                        
                        raw_text = page_data.get("main_text", "")
                        book_p = page_data.get("book_page_num", None)
                        class_code = page_data.get("classification_code", None)
                        
                        tg_ref_id = page_msg_ids.get(container_p, "N/A")
                        
                        if page_data.get("footnotes"): 
                            global_footnotes.append(f"[{container_p}] {page_data['footnotes']}")
                        
                        if page_data.get("marginalia"): 
                            cleaned_marg = page_data["marginalia"].strip()
                            if len(cleaned_marg) > 3: global_marginalia.add(cleaned_marg)
                        
                        final_page_text = raw_text.strip()
                        if class_code:
                            final_page_text = f"**{class_code}**\n\n" + final_page_text

                        global_text_blocks.append({
                            "container_idx": container_p,
                            "book_num": book_p,
                            "tg_ref": tg_ref_id, 
                            "text": final_page_text
                        })
                    
                    if global_text_blocks:
                        last_text_content = global_text_blocks[-1]["text"]
                        last_batch_tail_text = last_text_content[-1000:]
                    
                    for p in slice_paths:
                        if os.path.exists(p): os.remove(p)
                        
                    cursor += len(slice_paths)
                    
                except Exception as e:
                    log(f"      ⚠️ Failed: {str(e)[:100]}")
                    if "ResourceExhausted" in str(e) or "429" in str(e):
                        log("      ❌ CRITICAL API LIMIT. Stopping book.")
                        book_failed = True
                        break
                    
                    new_size = current_batch_size
                    for s in BATCH_STRATEGY:
                        if s < current_batch_size:
                            new_size = s
                            break
                    if new_size == current_batch_size:
                        cursor += 1 
                        current_batch_size = 1
                    else:
                        log(f"      ⬇️ Reducing batch size: {current_batch_size} -> {new_size}")
                        current_batch_size = new_size

            if book_failed:
                await client.send_message(dst_ent, f"❌ **Aborted:** {fname} (API Limits)")
                await update_last_id(client, src_ent, msg.id)
                continue

            # === GLOBAL ASSEMBLY ===
            final_md = f"# {fname}\n\n"
            
            final_md += "## Metadata\n"
            # Explicit order of ALL keys
            meta_order = [
                "isbn", "title", "authors", "co_authors", "publisher", "year", 
                "document_type", "historical_period_description", 
                "historical_year_start", "historical_year_end", "summary",
                "source_telegram_channel_id", "source_origin_msg_id"
            ]
            
            for k in meta_order:
                val = full_bibliographic_data.get(k, "")
                if isinstance(val, list):
                    val_str = ", ".join([str(x) for x in val if x]) # Filter empty strings
                else:
                    val_str = str(val) if val is not None else ""
                
                # Print key even if empty, but NO "N/A"
                final_md += f"- **{k}**: {val_str}\n"
            
            final_md += "\n## Content\n\n"
            
            joined_content = ""
            for i, block_data in enumerate(global_text_blocks):
                c_idx = block_data["container_idx"]
                b_num = block_data["book_num"]
                tg_id = block_data.get("tg_ref", "N/A")
                text = block_data["text"]
                
                label = f"> `[File: {c_idx} | Ref: {tg_id}]`"
                if b_num: label = f"> `[File: {c_idx} | Book: {b_num} | Ref: {tg_id}]`"
                
                page_marker = f"\n\n{label}\n"
                if i > 0: joined_content += page_marker + text
                else: joined_content += f"{label}\n" + text
            
            final_md += joined_content.strip()

            if global_resources:
                final_md += "\n\n## APPX: Resources\n"
                for res in global_resources:
                    final_md += f"- **ID**: `{res['id']}`\n"
                    final_md += f"  - **Caption**: {res['caption']}\n"
                    final_md += f"  - **Link**: {res['link']}\n"
            
            if global_footnotes:
                final_md += "\n\n## APPX: Footnotes\n" + "\n\n".join(global_footnotes)
            
            if global_marginalia:
                final_md += "\n\n## APPX: Marginalia\n" + "\n".join(sorted(list(global_marginalia)))

            md_out = os.path.join(TEMP_DIR, f"{os.path.splitext(fname)[0]}.md")
            with open(md_out, "w", encoding="utf-8") as f: f.write(final_md)
            await client.send_file(dst_ent, md_out, caption=f"📚 **MD FULL** (Tok: {t_in+t_out})", force_document=True)
            
            await mark_processed(client, msg)
            await update_last_id(client, src_ent, msg.id)
            log(f"   🎉 Done.")

        except Exception as e:
            log(f"❌ Error ID {curr_id}: {e}")
            import traceback; traceback.print_exc()
        
        finally:
            if os.path.exists(CROPS_DIR):
                shutil.rmtree(CROPS_DIR)
                os.makedirs(CROPS_DIR, exist_ok=True)
            if os.path.exists(TEMP_DIR):
                for f in os.listdir(TEMP_DIR):
                    if not f.endswith(".md"):
                        try: os.remove(os.path.join(TEMP_DIR, f))
                        except: pass
                        
        curr_id += 1; processed += 1; pbar.update(1)

    log(f"\n🏁 End. Total Tokens: {t_in}/{t_out}")
    await send_report(client, dst_ent)
    await client.disconnect()

# === RUN ===
print("🟡 Pre-flight check...")
try:
    _ = UserSecretsClient().get_secret("TELEGRAM_API_ID")
    print("🟢 Starting main_logic...")
    await main_logic()
except Exception as e: print(f"🔴 FAIL: {e}")

🧠 Models initialized. Main: models/gemini-2.5-flash. Delay: 30.0s.
🟡 Pre-flight check...
🟢 Starting main_logic...
🚀 **Launch V57.1 (Full Meta + JSON Repair):** 21:55:15
✅ Telegram OK.


Scan:   0%|          | 0/500 [00:00<?, ?msg/s]


📘 **Маркин_Д_Немцы_Калининграда_2009,_OCR.pdf** (ID: 67)
   🔄 Batch 1-4 (Size: 4)...
      📡 [models/gemini-2.5-flash] Sending...
      ✅ [models/gemini-2.5-flash] OK in 98.7s
   🎉 Done.
🏁 End.

🏁 End. Total Tokens: 1636/5564


In [3]:
# === ЯЧЕЙКА 3: V53 (MULTI-LANG SUPPORT & CONDITIONAL METADATA) ===

import os

import time

import json

import asyncio

import glob

import re

import google.generativeai as genai

from datetime import datetime

from kaggle_secrets import UserSecretsClient

from telethon import TelegramClient

from telethon.sessions import MemorySession



# === SETTINGS ===

INPUT_DIR = "./temp_processing"

OUTPUT_PREVIEW_FILE = "RAG_CHUNKS_PREVIEW.md"



# API Safety

GLOBAL_REQUEST_DELAY = 30.0  # 30 sec hard limit

TEXT_WINDOW_SIZE = 15000     



# Chunk Constraints

MAX_CHARS_PER_CHUNK = 2000

MAX_TOKENS_PER_CHUNK = 350



# Debug Limit

BOOKS_LIMIT = 1 



# === AUTH SETUP ===

user_secrets = UserSecretsClient()

try:

    GENAI_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

    genai.configure(api_key=GENAI_KEY)

    

    API_ID = int(user_secrets.get_secret("TELEGRAM_API_ID"))

    API_HASH = user_secrets.get_secret("TELEGRAM_API_HASH")

    BOT_TOKEN = user_secrets.get_secret("TELEGRAM_BOT_TOKEN")

    DEST_ID = int(user_secrets.get_secret("DEST_CHANNEL_ID"))

except:

    print("❌ Secrets missing")



MODEL_NAME = "models/gemini-2.5-flash"



# === UTILS ===

class RateLimiter:

    def __init__(self, delay):

        self.delay = delay

        self.last_call = 0

    

    async def wait(self):

        now = time.time()

        elapsed = now - self.last_call

        if elapsed < self.delay:

            wait_time = self.delay - elapsed + 1

            print(f"      ⏳ Pace: waiting {wait_time:.1f}s...")

            await asyncio.sleep(wait_time)

        self.last_call = time.time()



limiter = RateLimiter(GLOBAL_REQUEST_DELAY)



def get_flag_emoji(lang_code):

    flags = {

        "ru": "🇷🇺", "de": "🇩🇪", "en": "🇬🇧", 

        "pl": "🇵🇱", "fr": "🇫🇷", "it": "🇮🇹",

        "lt": "🇱🇹"

    }

    return flags.get(lang_code, "🌐")



def parse_processed_md(filepath):

    if not os.path.exists(filepath): return None, None

    with open(filepath, 'r', encoding='utf-8') as f: content = f.read()

    

    meta = {}

    meta_match = re.search(r'## Metadata\n(.*?)\n\n## Content', content, re.DOTALL)

    if meta_match:

        lines = meta_match.group(1).split('\n')

        for line in lines:

            if line.strip().startswith('- **'):

                match_row = re.search(r'\*\*(\w+)\*\*: (.*)', line)

                if match_row: meta[match_row.group(1)] = match_row.group(2).strip()

    

    text_match = re.search(r'## Content\n(.*?)(\n## APPX|$)', content, re.DOTALL)

    clean_text = text_match.group(1).strip() if text_match else ""

    return meta, clean_text



def create_smart_windows(text, max_size):

    windows = []

    current_window = ""

    paragraphs = text.split('\n\n')

    for para in paragraphs:

        if len(current_window) + len(para) < max_size:

            current_window += para + "\n\n"

        else:

            if current_window: windows.append(current_window)

            current_window = para + "\n\n"

    if current_window: windows.append(current_window)

    return windows



def get_chunking_prompt(book_meta, text_window):

    return f"""

    ROLE: Database Engineer for Vector Search.

    TASK: Split text into SEMANTIC CHUNKS and enrich metadata.

    

    METADATA:

    Title: {book_meta.get('title', 'Unknown')}

    

    CONSTRAINTS:

    1. Max Size: ~{MAX_CHARS_PER_CHUNK} chars (~{MAX_TOKENS_PER_CHUNK} tokens).

    2. **LANGUAGE RULES (STRICT)**: 

       - Detect the ACTUAL language of the chunk (e.g., 'de', 'ru', 'en', 'pl').

       - IF lang == 'de': Write `semantic_header` and `keywords` in **GERMAN**.

       - IF lang == 'ru': Write `semantic_header` and `keywords` in **RUSSIAN**.

       - IF lang is ANY OTHER (en, pl, etc): Write `semantic_header` and `keywords` in **RUSSIAN**.

    

    INPUT TEXT (with page markers):

    \"\"\"{text_window}\"\"\"

    

    INSTRUCTIONS:

    1. **Classification**:

       - `content` (narrative), `footnotes_definitions` (list of notes), `bibliography`.

    

    2. **Citation Detection**:

       - If markers like "[1]", "¹" found -> extract to `citation_ids`.

    

    3. **Context**:

       - Find nearest `> [File: X | Book: Y]` marker.

    

    4. **Enrichment**:

       - `semantic_header`: Summary (follow Language Rules!).

       - `keywords`: 3-5 tags (follow Language Rules!).

       - `images`: Extract `[[IMG:id]]`.

    

    OUTPUT FORMAT (JSON Array):

    [

      {{

        "chunk_content": "string",

        "chunk_type": "content",

        "lang": "code (ru, de, en...)",

        "container_page": int,

        "book_page": "string",

        "semantic_header": "string",

        "keywords": ["tag1"],

        "images": ["img_id"],

        "citation_ids": ["1"]

      }}

    ]

    """



async def process_rag_pipeline():

    all_md_files = sorted(glob.glob(os.path.join(INPUT_DIR, "*.md")))

    valid_files = [f for f in all_md_files if "RAG_CHUNKS" not in f]

    

    if not valid_files:

        print("❌ No source MD files found.")

        return



    files_to_process = valid_files[:BOOKS_LIMIT]

    print(f"📂 Found {len(valid_files)}. Processing: {len(files_to_process)}.")

    

    total_in = 0

    total_out = 0

    total_chunks = 0

    

    preview_output = f"# RAG DB PREVIEW (V53)\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n"

    

    client = TelegramClient(MemorySession(), API_ID, API_HASH)

    await client.start(bot_token=BOT_TOKEN)

    dest_ent = await client.get_entity(DEST_ID)

    

    for md_path in files_to_process:

        filename = os.path.basename(md_path)

        print(f"\n📘 Processing: {filename}")

        

        meta, full_text = parse_processed_md(md_path)

        if not full_text: continue

            

        windows = create_smart_windows(full_text, TEXT_WINDOW_SIZE)

        print(f"   ✂️ Windows: {len(windows)}")

        

        book_chunks = [] 

        

        for i, window in enumerate(windows):

            await limiter.wait()

            

            model = genai.GenerativeModel(

                model_name=MODEL_NAME,

                generation_config={"response_mime_type": "application/json", "temperature": 0.1}

            )

            

            try:

                print(f"      📡 Window {i+1}/{len(windows)}...")

                prompt = get_chunking_prompt(meta, window)

                

                response = await asyncio.to_thread(model.generate_content, prompt)

                

                if response.usage_metadata:

                    t_in = response.usage_metadata.prompt_token_count

                    t_out = response.usage_metadata.candidates_token_count

                    total_in += t_in

                    total_out += t_out

                    print(f"      ✅ OK (Tok: {t_in}/{t_out})")

                

                new_chunks = json.loads(response.text)

                

                for ch in new_chunks:

                    ch['meta_source'] = filename

                    ch['meta_isbn'] = meta.get('isbn')

                    ch['meta_title'] = meta.get('title')

                    book_chunks.append(ch)

                

            except Exception as e:

                print(f"      ❌ Error: {e}")

        

        total_chunks += len(book_chunks)

        

        preview_output += f"# Source: {filename}\n"

        preview_output += f"**Chunks**: {len(book_chunks)} | **ISBN**: {meta.get('isbn')}\n\n"

        

        for idx, ch in enumerate(book_chunks):

            lang_code = ch.get('lang', 'unknown')

            flag = get_flag_emoji(lang_code)

            ctype = ch.get('chunk_type', 'content').upper()

            

            preview_output += f"### [{idx+1}] {flag} {ctype}\n"

            

            loc = f"File: {ch.get('container_page')}"

            if ch.get('book_page'): loc += f" | Book: {ch.get('book_page')}"

            

            preview_output += f"- **Loc**: `{loc}`\n"

            preview_output += f"- **Topic**: {ch.get('semantic_header')}\n"

            preview_output += f"- **Tags**: {', '.join(ch.get('keywords', []))}\n"

            

            if ch.get('images'):

                preview_output += f"- **IMGS**: `{ch.get('images')}`\n"

            if ch.get('citation_ids'):

                preview_output += f"- **REFS**: {ch.get('citation_ids')}\n"

            

            preview_output += f"\n```text\n{ch.get('chunk_content')}\n```\n\n"

            

            db_rec = {

                "content": ch.get('chunk_content'),

                "vector_meta": {

                    "isbn": ch.get('meta_isbn'),

                    "lang": lang_code,

                    "type": ch.get('chunk_type'),

                    "page_doc": ch.get('container_page'),

                    "page_book": ch.get('book_page'),

                    "has_refs": bool(ch.get('citation_ids'))

                }

            }

            preview_output += f"> DB: `{json.dumps(db_rec, ensure_ascii=False)}`\n\n---\n"



    with open(OUTPUT_PREVIEW_FILE, "w", encoding="utf-8") as f:

        f.write(preview_output)

    

    summary = (

        f"📊 **RAG Chunking Report**\n"

        f"Books: {len(files_to_process)}\n"

        f"Chunks: {total_chunks}\n"

        f"Tokens In: {total_in}\n"

        f"Tokens Out: {total_out}"

    )

    print(f"\n{summary}")

    

    try:

        await client.send_message(dest_ent, summary)

        await client.send_file(dest_ent, OUTPUT_PREVIEW_FILE, caption="🧠 **RAG Chunks Preview (V53)**")

    except Exception as e:

        print(f"TG Upload Error: {e}")

        

    await client.disconnect()



# === RUN ===

await process_rag_pipeline()

📂 Found 1. Processing: 1.

📘 Processing: Маркин_Д_Немцы_Калининграда_2009,_OCR.md
   ✂️ Windows: 2
      📡 Window 1/2...
      ✅ OK (Tok: 4172/5030)
      📡 Window 2/2...
      ✅ OK (Tok: 1249/1206)

📊 **RAG Chunking Report**
Books: 1
Chunks: 12
Tokens In: 5421
Tokens Out: 6236
